<a href="https://colab.research.google.com/github/digdemorhan/Keras_With_Examples_For_Deep_Learning/blob/main/ModelSubclassingWithKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Verisetinin alınması
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

In [3]:
#Değişkenin tag'lerinin görüntülenmesi
for i in housing:
  print(i)

data
target
frame
target_names
feature_names
DESCR


In [4]:
#Verisetinin eğitim ve test şeklinde parçalanması
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)

In [5]:
#Validation parametrelerinin oluşturulması
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [7]:
#Modelin mimarisinin oluşturulması
import tensorflow as tf
from tensorflow import keras

class WideAndDeepModel(tf.keras.Model):
  def __init__(self, units=30, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.norm_layer_wide = tf.keras.layers.Normalization()
    self.norm_layer_deep = tf.keras.layers.Normalization()
    self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
    self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
    self.main_output = tf.keras.layers.Dense(1)

  def call(self, inputs):
    input_wide, input_deep = inputs
    norm_wide = self.norm_layer_wide(input_wide)
    norm_deep = self.norm_layer_deep(input_deep)
    hidden1 = self.hidden1(norm_deep)
    hidden2 = self.hidden2(hidden1)
    concat = tf.keras.layers.concatenate([norm_wide, hidden2])
    output = self.main_output(concat)
    return output

In [8]:
#Modelden örnek alınması
tf.random.set_seed(42)
model = WideAndDeepModel(30, activation="relu", name="wide_and_deep")

In [9]:
#Modelin compile edilmesi
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)
model.compile(
    loss = "mse",
    optimizer = optimizer,
    metrics = ["RootMeanSquaredError"]
)

In [10]:
#Modelin girdilerinin ayarlanması
X_train_wide, X_train_deep = X_train[:,:5], X_train[:,2:]
X_valid_wide, X_valid_deep = X_valid[:,:5], X_valid[:,2:]
X_test_wide, X_test_deep = X_test[:,:5], X_test[:,2:]

In [11]:
#Modelin eğitimi
model.norm_layer_deep.adapt(X_train_deep)
model.norm_layer_wide.adapt(X_train_wide)

history = model.fit(
    (X_train_wide, X_train_deep),
    y_train,
    validation_data = ((X_valid_wide, X_valid_deep), y_valid),
    epochs = 10,
)

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - RootMeanSquaredError: 1.5100 - loss: 2.3753 - val_RootMeanSquaredError: 0.8675 - val_loss: 0.7525
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.7595 - loss: 0.5778 - val_RootMeanSquaredError: 0.7634 - val_loss: 0.5828
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.6791 - loss: 0.4616 - val_RootMeanSquaredError: 0.6994 - val_loss: 0.4891
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.6529 - loss: 0.4264 - val_RootMeanSquaredError: 0.8815 - val_loss: 0.7770
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - RootMeanSquaredError: 0.6364 - loss: 0.4052 - val_RootMeanSquaredError: 0.9453 - val_loss: 0.8936
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6242 - loss: 0.3897 - val_RootMeanSquaredError: 0.9611 - val_loss: 0.9238
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6159 - los

In [12]:
#Modelin değerlendirilmesi
eval_results = model.evaluate((X_test_wide, X_test_deep),y_test)
eval_results

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5919 - loss: 0.3504


[0.34819620847702026, 0.590081512928009]

In [13]:
#Model tahmini
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]
y_pred = model.predict((X_new_wide, X_new_deep))
y_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step


array([[0.4641111],
       [1.6061474],
       [3.58684  ]], dtype=float32)

In [14]:
#Gerçek değerlerle karşılaştırılması
y_test[:3]

array([0.477  , 0.458  , 5.00001])